In [ ]:
import numpy as np
import pandas as pd
import json
import textdistance
import random
import math
from rdkit.Chem import AllChem as Chem
from rdkit import DataStructs

from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Dropout, Flatten, Activation, Conv1D
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model
import keras.backend as K
import tensorflow as tf
from nested_lstm import NestedLSTM

In [ ]:
GENERATOR_MODEL_PATH = './model/pre_trained/generator-loss0.2505-acc0.8949-val_loss3.4282-val_acc0.5270.hdf5'
GENERATOR_DATASET = './dataset/generator_dataset.txt'

PREDICTOR_MODEL_PATH = './model/pre_trained/predictor-MSE_val_loss-0.1881.hdf5'
PREDICTOR_STATS_JSON = './dataset/predictor_dataset_stats.json'
PREDICTOR_DATASET = './dataset/predictor_dataset.csv'

RADIUS = 3
NBITS = 2048

#### Generator

In [ ]:
raw_text = open(GENERATOR_DATASET).read()
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_vocab = len(chars)
n_chars = len(raw_text)

seq_length = 5
dataX = []
dataY = []
for i in range(0, len(raw_text) - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    if seq_out != "\n":
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(1024, input_shape=(5, 1), return_sequences=True))
model_lstm.add(NestedLSTM(1024, depth=4, dropout=0.1, recurrent_dropout=0.0, return_sequences=True))
model_lstm.add(LSTM(1024, return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(Activation('relu'))
model_lstm.add(LSTM(512, return_sequences=True))
model_lstm.add(Dropout(0.1))
model_lstm.add(Activation('relu'))
model_lstm.add(LSTM(512))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(23, activation='softmax'))

In [ ]:
model_lstm.load_weights(GENERATOR_MODEL_PATH)
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
def random_seed():
    test_seed = dataX[np.random.randint(0, len(dataX)-1)]
    while 0 in test_seed:
        test_seed = dataX[np.random.randint(0, len(dataX)-1)]
    return test_seed

def generate_seed():    
    pattern = random_seed()
    seq = [int_to_char[value] for value in pattern]

    for i in range(25):
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model_lstm.predict(x, verbose=0)
        index = np.argmax(prediction)
        if index != 0:
            result = int_to_char[index]
            seq += result
            pattern.append(index)
            pattern = pattern[1:len(pattern)]
        else:
            pattern = random_seed()

    return ''.join(seq)

Test

In [ ]:
generate_seed()

#### Predictor

In [ ]:
with open(PREDICTOR_STATS_JSON) as f:
    dict_data = json.load(f)
    
model_cnn = load_model(PREDICTOR_MODEL_PATH)

In [ ]:
class Fingerprint_Generation:
    def __init__(self, smiles,radius=RADIUS,nbits=NBITS):
        self.lookupfps = {}
        
        for key, value in lookupsmiles.items():
            mol = Chem.MolFromSmiles(value)
            fp = np.array(Chem.GetMorganFingerprintAsBitVect(mol,radius,nbits))
            self.lookupfps[key] = fp
        self.lookupfps[' '] = np.zeros(self.lookupfps['A'].shape)
    
    def seq(self, seq):
        fp = np.asarray([self.lookupfps[seq[i]] for i in range(len(seq))])
        return fp

In [ ]:
lookupsmiles = {
         '2': 'NC(CSC1=C(F)C(F)=C(C(F)=C1F)C1=C(F)C(F)=C(SCC(N)C(N)=O)C(F)=C1F)C(N)=O',
         '3': 'CC(=O)CC1=CN(CCCCC(N)C(N)=O)N=N1',
         'A': 'N[C@@H](C)C(O)=O',
         'B': 'C(CN)C(=O)O',
         'X': 'C(CCC(=O)O)CCN',
         'R': 'N[C@@H](CCCNC(N)=N)C(O)=O', 
         'N': 'N[C@@H](CC(N)=O)C(O)=O', 
         'D': 'N[C@@H](CC(O)=O)C(O)=O', 
         'C': 'N[C@H](C(O)=O)CS', 
         'E': 'N[C@@H](CCC(O)=O)C(O)=O', 
         'Q': 'N[C@@H](CCC(N)=O)C(O)=O', 
         'G': 'NCC(O)=O', 
         'H': 'N[C@@H](CC1=CNC=N1)C(O)=O', 
         'I': 'N[C@@H]([C@@H](C)CC)C(O)=O', 
         'L': 'N[C@@H](CC(C)C)C(O)=O', 
         'K': 'N[C@@H](CCCCN)C(O)=O', 
         'M': 'N[C@@H](CCSC)C(O)=O', 
         'F': 'N[C@@H](CC1=CC=CC=C1)C(O)=O', 
         'P': 'O=C(O)[C@H]1NCCC1', 
         'S': 'N[C@@H](CO)C(O)=O', 
         'T': 'N[C@@H]([C@H](O)C)C(O)=O', 
         'W': 'N[C@@H](CC1=CNC2=C1C=CC=C2)C(O)=O', 
         'Y': 'N[C@@H](CC1=CC=C(O)C=C1)C(O)=O', 
         'V': 'N[C@@H](C(C)C)C(O)=O',
         '@': 'N[C@@H](CSC1=C(C(F)=C(C(F)=C1F)C2=C(C(F)=C(C(F)=C2F)SC[C@@H](C(O)=O)N)F)F)C(O)=O',
         '#': 'N[C@H](C(O)=O)CSC1=CC(SC[C@@H](N)C(O)=O)=CC(SC[C@H](N)C(O)=O)=C1'
}

fp = Fingerprint_Generation(lookupsmiles)

In [ ]:
def predict(sequence, model):
    features_max = 108
    fp_seq = fp.seq(sequence)
    n_rows = features_max - len(sequence)
    shape_padding = (n_rows, NBITS)
    padding_array = np.zeros(shape_padding)
    fp_seq = np.concatenate((fp_seq, padding_array), axis = 0)
    
    return model.predict(np.asarray([fp_seq]))[0][0]

Test

In [ ]:
predict('AA', model_cnn) # Note: Intensity is scaled.

#### Optimizer

In [ ]:
ORIGINAL_AAS = [i for i in lookupsmiles.keys() if i not in '@#1234567890']
MAX_LEN = 108

SEQ_LIST = pd.read_csv(PREDICTOR_DATASET)['sequences'].tolist()

In [ ]:
def net_charge(sequence):
    #http://www.chem.ucalgary.ca/courses/351/Carey5th/Ch27/ch27-1-4-2.html
    acidic = [sequence.count('D'), sequence.count('E'), sequence.count('C'), sequence.count('Y')]
    basic = [sequence.count('R'), sequence.count('K'), sequence.count('H')]

    acidic_pKa = [math.pow(10, 3.65), math.pow(10, 4.25), math.pow(10, 8.37), math.pow(10, 10.46)]
    basic_pKa = [math.pow(10, 10.76), math.pow(10, 9.74), math.pow(10, 7.59)]

    basic_coeff = [x*(1/(x+math.pow(10, 7))) for x in basic_pKa]
    acidic_coeff = [math.pow(10, 7)/(x+math.pow(10, 7)) for x in acidic_pKa]

    charge = - sum(np.multiply(acidic_coeff, acidic)) + sum(np.multiply(basic_coeff, basic))
    return charge

In [ ]:
def deletion(str_sequence):
    str_sequence = str_sequence.split(" ")[0]
    toremove = random.randint(0, len(str_sequence) - 1)
    new_str = str_sequence[:toremove] + str_sequence[toremove+1:] + " "
    return new_str

def insertion(str_sequence, aminoacids=ORIGINAL_AAS):
    str_sequence = str_sequence.split(" ")[0]
    toinsert = random.randint(0, len(str_sequence))
    new_str = str_sequence[:toinsert] + random.choice(aminoacids) + str_sequence[toinsert:]
    return new_str[:MAX_LEN]

def swap(str_sequence, aminoacids=ORIGINAL_AAS):
    existingaas = [i for i in set(str_sequence) if i != " "]
    aatoreplace = random.choice(existingaas)
    aaindices = [index for index, value in enumerate(str_sequence) if value == aatoreplace]
    indextoreplace = random.choice(aaindices)
    new_str = str_sequence[:indextoreplace] + random.choice(aminoacids) + str_sequence[indextoreplace+1:]
    return new_str

def hybrid(str_sequence, hybrid_list = SEQ_LIST):
    str_sequence = str_sequence.split(" ")[0]
    tohybrid = random.randint(0, len(str_sequence)-1)
    hybridlen = random.randint(0, int((len(str_sequence) - tohybrid)/5))+1
    hybrid_from = random.choice(hybrid_list)
    leastlen = max(len(min(hybrid_list, key=len)), len(hybrid_from))
    while leastlen < hybridlen:
        hybrid_from = random.choice(hybrid_list)
        leastlen = len(hybrid_from)
        print (leastlen, hybridlen)
    index_hybrid_max = leastlen-hybridlen
    if index_hybrid_max > 0:
        index_hybrid_max = index_hybrid_max - 1
    index_hybrid = random.randint(0, index_hybrid_max)
    hybrid_from = hybrid_from[index_hybrid:index_hybrid+hybridlen]
    new_str = str_sequence[:tohybrid] + hybrid_from + str_sequence[tohybrid+hybridlen:]
    return new_str

In [ ]:
mutation_list = [insertion, deletion, swap, hybrid]

In [ ]:
def fitnessfunc(sequence, i):
    nn_pred = predict(sequence, model_cnn)
    arg_count = (((sequence).count('R')) - dict_data['mean_R_count']) / dict_data['std_R_count']
    len_count = (len(sequence) - dict_data['mean_len_seq'])/dict_data['std_len_seq']
    charge = (net_charge(sequence) - dict_data['mean_charge'])/dict_data['std_charge']
    
    similarity_training = max([textdistance.jaro_winkler.similarity(sequence, reference)
                               for reference in SEQ_LIST])
    
    max_similarity_predicted = 0
    similarity_predicted = 0
    
    try:
        for k in range(1, i+1):
            predicted_sequences = list(Seq_df.at[k, 'new_dict'].keys())
            similarity_predicted = max(np.asarray([
                textdistance.jaro_winkler.similarity(sequence, predicted_sequences[j])
                                              for j in range(len(predicted_sequences))]))
            max_similarity_predicted = max(similarity_predicted, max_similarity_predicted)
    except:
        pass

    similarity = max(similarity_training, max_similarity_predicted)
    
    value =  0.5*nn_pred - 0.5*(
        0.5*arg_count + 0.2*len_count - 0.1*charge + similarity)
    
    return value, nn_pred, arg_count, len_count, charge, similarity

In [ ]:
def genetic_algorithm(sequence, i,
                      max_attempts,
                      fitnessfunc=fitnessfunc,
                      mutation_list=mutation_list):
    
    T = 100 # Parameter for simulated annealing
    
    oldseq = sequence
    for attempt in range(max_attempts):
        mutation = random.choice(mutation_list)
        oldvalue, nn_pred, arg_count, len_count, charge, similarity = fitnessfunc(oldseq, i)
        newseq = mutation(oldseq)
        newvalue, nn_pred, arg_count, len_count, charge, similarity = fitnessfunc(newseq, i)
        
        delta = newvalue - oldvalue
        
        if (newvalue * np.exp(-delta/T)) > oldvalue:
            oldseq = newseq
            Seq_df.at[i, 'new_dict'][newseq] = [newvalue, nn_pred, arg_count, len_count, charge, similarity]
        else:
            continue

#### Generator -> (Predictor-Optimizer)

Generating Seeds

In [ ]:
number_seeds = 2

Seq_df = pd.DataFrame(columns=['seed', 'new_dict'])

for counter in range(number_seeds):
    Seq_df.at[counter, 'seed'] = generate_seed()
    Seq_df.at[counter, 'new_dict'] = {}

Predictor-Optimizer Loop

In [ ]:
max_attempts = 10
new_seq_dict = {}

for i in range(Seq_df.shape[0]):
    genetic_algorithm(Seq_df.at[i, 'seed'], i, max_attempts)
    new_seq_dict.update(Seq_df.at[i, 'new_dict'].items())

In [ ]:
df_temp = pd.DataFrame.from_dict(new_seq_dict, orient='index')
df_temp = df_temp.rename(columns = {
    0:'value', 
    1:'norm_intensity', 
    2:'norm_arg_count', 
    3:'norm_len_count', 
    4:'charge',
    5:'similarity'}
                              )
df_temp['arg_count'] = df_temp.index.str.count('R')
df_temp['sequences'] = df_temp.index
df_temp['net_charge'] = df_temp['sequences'].apply(net_charge)
df_temp['len'] = df_temp.index.map(len)

In [ ]:
ga_df = pd.DataFrame({
    'sequences':df_temp['sequences'].tolist(),
    'intensity':(df_temp['norm_intensity']*dict_data['std_intensity'] + 
                                dict_data['mean_intensity']).tolist(),
    'length':(df_temp['len']).tolist(),
    'relative_Arg':(df_temp['arg_count']/df_temp['len']).tolist(),
    'relative_charge':(df_temp['net_charge']/df_temp['len']).tolist(),
})

In [ ]:
ga_df